# Datast 6 : Wine Quality

### Importing the libraries 

In [1]:
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
import time
import psutil
import os
import numpy as np
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, accuracy_score, f1_score

### Importinng the PlumberOptimizer

In [2]:
import sys
sys.path.append('../')
from Plumber import PlumberOptimizer

### Loading the Data and performing feature engineering:

In [3]:
# Wine Quality Dataset - Data Loading and Preprocessing
# Load red wine data from UCI repository
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
df = pd.read_csv(url, sep=';')

# Features and target
X = df.drop(columns=["quality"])
y = df["quality"]

# Transform target into a classification problem (e.g., group into 3 classes: low, medium, high)
y = y.apply(lambda q: 0 if q <= 5 else (1 if q == 6 else 2))

print("Shape of X:", X.shape)
print("Target class distribution:\n", y.value_counts())
X.head()


Shape of X: (1599, 11)
Target class distribution:
 quality
0    744
1    638
2    217
Name: count, dtype: int64


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Training set shape:", X_train.shape)
print("Test set shape:", X_test.shape)


Training set shape: (1279, 11)
Test set shape: (320, 11)


### Testing the performance and efficiency of the PlumberOptimizer

In [5]:
# Arrays to store execution times, memory usage, CPU usage, and evaluatin scores
plumber_time = np.zeros(5,'float')
plumber_mem = np.zeros(5,'float')
plumber_cpu =  np.zeros(5,'float')
plumber_score = np.zeros(5,'float')
for i in range(5):
    opt = PlumberOptimizer(X_train, y_train)

    # Monitor the current process, record the start time, initial CPU usage, and memory usage in MB
    process = psutil.Process(os.getpid())
    start_time = time.time()
    start_cpu = process.cpu_percent(interval=None)
    start_mem = process.memory_info().rss / (1024 * 1024) 


    best = opt.optimize()

    end_time = time.time()
    end_cpu = process.cpu_percent(interval=None)
    end_mem = process.memory_info().rss / (1024 * 1024)  

    # Calculating execution time, memory usage, and CPU usage
    plumber_time[i] = end_time - start_time
    plumber_mem[i] = end_mem - start_mem
    plumber_cpu[i] = end_cpu - start_cpu

    # Training and evaluating the model using the best parameters
    rf = RandomForestClassifier(**best , random_state=42 )
    rf.fit(X_train, y_train)

    y_pred = rf.predict(X_test)


    plumber_score[i] = accuracy_score(y_test, y_pred)

# summary for the Plumber Optimizer
print("\n" *3)
print("*************")
print(f"Execution Time: mean={np.mean(plumber_time):.2f} seconds, std={np.std(plumber_time)} seconds, max={np.max(plumber_time)} seconds, min={np.min(plumber_time)} seconds")
print(f"CPU Usage: mean={np.mean(plumber_cpu):.2f}%, std={np.std(plumber_cpu):.2f}%, max={np.max(plumber_cpu):.2f}%, min={np.min(plumber_cpu):.2f}%")
print(f"Memory Usage: mean={np.mean(plumber_mem):.2f} MB, std={np.std(plumber_mem):.2f} MB, max={np.max(plumber_mem):.2f} MB, min={np.min(plumber_mem):.2f} MB")
print(f"Validation Score: mean={np.mean(plumber_score)} , std={np.std(plumber_score)} , max={np.max(plumber_score)} , min={np.min(plumber_score)} ")

A single model takes  0.1495380401611328 seconds to run
Suggested number of trails is 149 
Best Parameters: {'n_estimators': 300, 'max_depth': 30, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'log2'}
Best Score: 0.6903772360941606
A single model takes  0.12938594818115234 seconds to run
Suggested number of trails is 149 
Best Parameters: {'n_estimators': 300, 'max_depth': 25, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'log2'}
Best Score: 0.6903809010712728
A single model takes  0.12839770317077637 seconds to run
Suggested number of trails is 149 
Best Parameters: {'n_estimators': 200, 'max_depth': 35, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'log2'}
Best Score: 0.6911542112419508
A single model takes  0.1293330192565918 seconds to run
Suggested number of trails is 149 
Best Parameters: {'n_estimators': 150, 'max_depth': 30, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt'}
Best Score: 0.6927319838887606
A 

### Testing the performance and efficiency of the GridSearch

In [6]:
# Defining GridSearch parameters
param_grid = {
    'n_estimators': [100, 200, 300, 400, 500],  
    'max_features': ['sqrt', 'log2'],  
    'max_depth': [10, 20, 30, 40, 50],  
    'min_samples_split': [2, 5, 10, 12, 16, 20], 
    'min_samples_leaf': [2, 5, 6, 8,12,16, 20]  
}

rf = RandomForestClassifier(random_state=42)

grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, n_jobs=-1, scoring='accuracy')

# Monitor the current process, record the start time, initial CPU usage, and memory usage in MB
process = psutil.Process(os.getpid())
start_time = time.time()
start_cpu = process.cpu_percent(interval=None)
start_mem = process.memory_info().rss / (1024 * 1024)  

grid_search.fit(X_train, y_train)

end_time = time.time()
end_cpu = process.cpu_percent(interval=None)
end_mem = process.memory_info().rss / (1024 * 1024) 

# Print metrics for GridSearch
print(f"Execution Time: {end_time - start_time:.2f} seconds")
print(f"CPU Usage: {end_cpu - start_cpu:.2f}%")
print(f"Memory Usage: {end_mem - start_mem:.2f} MB")


# Training the model with the best parameters and evaluate
rf = RandomForestClassifier(** grid_search.best_params_, random_state=42 )
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Validation Accuracy: {accuracy}")
print(grid_search.best_params_)




/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Execution Time: 199.64 seconds
CPU Usage: 2.30%
Memory Usage: 7.91 MB
Validation Accuracy: 0.740625
{'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 100}


### Testing the performance and efficiency of the RandomSearch

In [7]:
# Defining the parameter for RandomizedSearch
param_dist = {
    'n_estimators': np.arange(50, 500),
    'max_features': ['sqrt', 'log2'],
    'max_depth': np.arange(10, 31),
    'min_samples_split': np.arange(2, 50),
    'min_samples_leaf': np.arange(1, 50)
}
# Arrays to store execution times, memory usage, CPU usage, and evaluation scores
rand_time = np.zeros(5,'float')
rand_mem = np.zeros(5,'float')
rand_cpu =  np.zeros(5,'float')
rand_score = np.zeros(5,'float')
for i in range(5):
    rf = RandomForestClassifier(random_state=42)
    random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_dist,
                                    n_iter=250, cv=3, n_jobs=-1)
    
    # Monitor the current process, record the start time, initial CPU usage, and memory usage in MB
    process = psutil.Process(os.getpid())
    start_time = time.time()
    start_cpu = process.cpu_percent(interval=None)
    start_mem = process.memory_info().rss / (1024 * 1024)  

    random_search.fit(X_train, y_train)

    end_time = time.time()
    end_cpu = process.cpu_percent(interval=None)
    end_mem = process.memory_info().rss / (1024 * 1024)  

    # Calculating execution time, memory usage, and CPU usage
    rand_time[i] = end_time - start_time
    rand_mem[i] = end_mem - start_mem
    rand_cpu[i] = end_cpu - start_cpu

    # Training the model with the best parameters and evaluate
    rf = RandomForestClassifier(** random_search.best_params_ , random_state=42 )
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)


    rand_score[i] = accuracy_score(y_test, y_pred)

#RandomizedSearchCV results
print(f"Execution Time: mean={np.mean(rand_time):.2f} seconds, std={np.std(rand_time)} seconds, max={np.max(rand_time)} seconds, min={np.min(rand_time)} seconds")
print(f"CPU Usage: mean={np.mean(rand_cpu):.2f}%, std={np.std(rand_cpu):.2f}%, max={np.max(rand_cpu):.2f}%, min={np.min(rand_cpu):.2f}%")
print(f"Memory Usage: mean={np.mean(rand_mem):.2f} MB, std={np.std(rand_mem):.2f} MB, max={np.max(rand_mem):.2f} MB, min={np.min(rand_mem):.2f} MB")
print(f"Validation Score: mean={np.mean(rand_score)} , std={np.std(rand_score)} , max={np.max(rand_score)} , min={np.min(rand_score)} ")


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Execution Time: mean=19.40 seconds, std=0.32063128585936984 seconds, max=19.93774724006653 seconds, min=18.983601808547974 seconds
CPU Usage: mean=5.06%, std=0.63%, max=5.70%, min=4.30%
Memory Usage: mean=2.17 MB, std=2.45 MB, max=6.28 MB, min=0.00 MB
Validation Score: mean=0.7474999999999999 , std=0.004592793267718473 , max=0.753125 , min=0.740625 


### Testing the performance and efficiency of the BayesSearchCV

In [8]:
# Define the parameter for BayesSearchCV
param_dist = {
    'n_estimators': Integer(50, 500),
    'max_features': Categorical(['sqrt', 'log2']),
    'max_depth': Integer(10, 50),
    'min_samples_split': Integer(2, 50),
    'min_samples_leaf': Integer(1, 50)
}
# Arrays to store execution times, memory usage, CPU usage, and evaluation scores
rand_time = np.zeros(5,'float')
rand_mem = np.zeros(5,'float')
rand_cpu =  np.zeros(5,'float')
rand_score = np.zeros(5,'float')
for i in range(5):
    print(i)
    rf = RandomForestClassifier(random_state=42)
    bayes_search = BayesSearchCV(estimator=rf, search_spaces=param_dist,
                             n_iter=70, cv=3, n_jobs=-1)

    # Monitor the current process, record the start time, initial CPU usage, and memory usage in MB
    process = psutil.Process(os.getpid())
    start_time = time.time()
    start_cpu = process.cpu_percent(interval=None)
    start_mem = process.memory_info().rss / (1024 * 1024) 

    bayes_search.fit(X_train, y_train)

    end_time = time.time()
    end_cpu = process.cpu_percent(interval=None)
    end_mem = process.memory_info().rss / (1024 * 1024)  

    # Calculating execution time, memory usage, and CPU usage
    rand_time[i] = end_time - start_time
    rand_mem[i] = end_mem - start_mem
    rand_cpu[i] = end_cpu - start_cpu

    # Training the model with the best parameters and evaluate
    rf = RandomForestClassifier(** bayes_search.best_params_ , random_state=42 )
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)


    rand_score[i] = accuracy_score(y_test, y_pred)


# BayesSearchCV results
print(f"Execution Time: mean={np.mean(rand_time):.2f} seconds, std={np.std(rand_time)} seconds, max={np.max(rand_time)} seconds, min={np.min(rand_time)} seconds")
print(f"CPU Usage: mean={np.mean(rand_cpu):.2f}%, std={np.std(rand_cpu):.2f}%, max={np.max(rand_cpu):.2f}%, min={np.min(rand_cpu):.2f}%")
print(f"Memory Usage: mean={np.mean(rand_mem):.2f} MB, std={np.std(rand_mem):.2f} MB, max={np.max(rand_mem):.2f} MB, min={np.min(rand_mem):.2f} MB")
print(f"Validation Score: mean={np.mean(rand_score)} , std={np.std(rand_score)} , max={np.max(rand_score)} , min={np.min(rand_score)} ")


0
1
2
3
4
Execution Time: mean=104.49 seconds, std=1.7504239386916904 seconds, max=106.46674275398254 seconds, min=101.42866015434265 seconds
CPU Usage: mean=960.76%, std=15.07%, max=985.40%, min=947.80%
Memory Usage: mean=70.86 MB, std=132.11 MB, max=334.66 MB, min=0.02 MB
Validation Score: mean=0.745 , std=0.006123724356957924 , max=0.75625 , min=0.7375 
